In [4]:
#import the dependinces
import pandas as pd
import csv
import numpy as np
import gmaps
import datetime
import time
from scipy import stats
from matplotlib import pyplot as plt
import os
import warnings

# Ignore unnecessary gmaps warning
warnings.filterwarnings('ignore')

In [11]:
#Import the csvs
lapd = pd.read_csv("../crime_data/LA_Crime_Data_from_2010_to_2019.csv")
arrests_df = pd.read_csv("../crime_data/NYPD_Arrests_Data__Historic_.csv")

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111674,190608903,03/28/2019 12:00:00 AM,03/28/2019 12:00:00 AM,400,6,Hollywood,644,1,648,ARSON,...,IC,Invest Cont,648.0,NaN,NaN,NaN,1400 N LA BREA AV,NaN,34.0962,-118.3490
2111675,190715222,08/15/2019 12:00:00 AM,08/14/2019 12:00:00 AM,1810,7,Wilshire,701,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),...,IC,Invest Cont,331.0,NaN,NaN,NaN,WILLOUGHBY AV,ORLANDO AV,34.0871,-118.3732
2111676,192004409,01/06/2019 12:00:00 AM,01/06/2019 12:00:00 AM,2100,20,Olympic,2029,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,...,IC,Invest Cont,930.0,NaN,NaN,NaN,6TH,VIRGIL,34.0637,-118.2870
2111677,191716777,10/17/2019 12:00:00 AM,10/16/2019 12:00:00 AM,1800,17,Devonshire,1795,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,17200 NAPA ST,NaN,34.2266,-118.5085


In [12]:
#Drop unessacary rows for LA
lapd.drop(columns=['Status Desc','DR_NO', 'Date Rptd','AREA ','Rpt Dist No', 'Part 1-2', 'Crm Cd','Mocodes','Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd','Weapon Used Cd','Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4','LOCATION', 'Cross Street'],axis=1,inplace=True)
lapd.columns

Index(['DATE OCC', 'TIME OCC', 'AREA NAME', 'Crm Cd Desc', 'Premis Desc',
       'Weapon Desc', 'LAT', 'LON'],
      dtype='object')

In [14]:
#Rename each crime committed in LA data
lapd['Crm Cd Desc'] = lapd['Crm Cd Desc'].replace({'VIOLATION OF COURT ORDER':'VIOLATION OF',
'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)':'VANDALISM', 'RAPE, ATTEMPTED':'RAPE',
'SHOPLIFTING - PETTY THEFT ($950 & UNDER)':'SHOPLIFTING','BURGLARY FROM VEHICLE':'BURGLARY',
'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT':'ASSAULT','THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD':'THEFT',
'BATTERY - SIMPLE ASSAULT':'BATTERY','CHILD NEGLECT (SEE 300 W.I.C.)':'CHILD','INTIMATE PARTNER - AGGRAVATED ASSAULT':'ASSAULT',
'INTIMATE PARTNER - SIMPLE ASSAULT':'ASSAULT','THEFT PLAIN - PETTY ($950 & UNDER)':'THEFT', 'ATTEMPTED ROBBERY':'ROBBERY',
'VANDALISM - MISDEAMEANOR ($399 OR UNDER)':'VANDALISM','RAPE, FORCIBLE':'RAPE','THROWING OBJECT AT MOVING VEHICLE':'VEHICLE',
'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)':'SHOPLIFTING','CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT':'CHILD',
'SHOTS FIRED AT INHABITED DWELLING':'SHOTS FIRED','THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)':'THEFT',
'VEHICLE - STOLEN':'VEHICLE','KIDNAPPING - GRAND ATTEMPT':'KIDNAPPING','THEFT PLAIN - ATTEMPT':'THEFT','THEFT, PERSON':'THEFT',
'VEHICLE - ATTEMPT STOLEN':'VEHICLE','BATTERY WITH SEXUAL CONTACT':'BATTERY','BATTERY POLICE (SIMPLE)':'BATTERY',
'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT':'CHILD','OTHER ASSAULT':'ASSAULT', 'VIOLATION OF RESTRAINING ORDER':'VIOLATION OF',
'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)':'CHILD','THEFT OF IDENTITY':'THEFT','ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER':'ASSAULT',
'CHILD ANNOYING (17YRS & UNDER)':'CHILD','SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH':'RAPE',
'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)':'THEFT','LETTERS, LEWD  -  TELEPHONE CALLS, LEWD':'LEWD CONDUCT',
'INDECENT EXPOSURE':'LEWD CONDUCT','BURGLARY FROM VEHICLE, ATTEMPTED':'BURGLARY','BURGLARY, ATTEMPTED':'BURGLARY',
'THREATENING PHONE CALLS/LETTERS':'CRIMINAL THREATS','SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ':'RAPE',                                                  
'BUNCO, PETTY THEFT':'BUNCO','VIOLATION OF TEMPORARY RESTRAINING ORDER':'VIOLATION OF','BUNCO, GRAND THEFT':'BUNCO',                                                   
'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)':'EMBEZZLEMENT','SEXUAL PENETRATION W/FOREIGN OBJECT':'RAPE','EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)':'EMBEZZLEMENT',                                                   
'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $400 & UNDER':'THEFT','CREDIT CARDS, FRAUD USE ($950.01 & OVER)':'FRAUD','DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $400':'THEFT',                                                  
'THEFT FROM PERSON - ATTEMPT':'THEFT','DRUNK ROLL - ATTEMPT':'DRUNK ROLL','SHOPLIFTING - ATTEMPT':'SHOPLIFTING',                                                   
'DISHONEST EMPLOYEE - GRAND THEFT':'THEFT','CREDIT CARDS, FRAUD USE ($950 & UNDER':'FRAUD','DOCUMENT WORTHLESS ($200.01 & OVER)':'DOCUMENT WORTHLESS',                                                   
'CHILD STEALING':'CHILD','THEFT FROM MOTOR VEHICLE - ATTEMPT':'THEFT', 'BATTERY ON A FIREFIGHTER':'BATTERY','THEFT, COIN MACHINE - PETTY ($950 & UNDER)':'THEFT',                                                  
'GRAND THEFT / INSURANCE FRAUD':'THEFT','BUNCO, ATTEMPT':'BUNCO','THEFT, COIN MACHINE - GRAND ($950.01 & OVER)':'THEFT',                                                   
'PURSE SNATCHING - ATTEMPT':'PURSE SNATCHING','SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT':'SHOTS FIRED','DISCHARGE FIREARMS/SHOTS FIRED':'SHOTS FIRED',                                                  
'CHILD ABANDONMENT':'CHILD','THEFT, COIN MACHINE - ATTEMPT':'THEFT','DISHONEST EMPLOYEE - PETTY THEFT':'THEFT','PETTY THEFT - AUTO REPAIR':'THEFT',                                                  
'TILL TAP - ATTEMPT':'TILL TAP','TILL TAP - PETTY ($950 & UNDER)':'TILL TAP','TILL TAP - GRAND THEFT ($950.01 & OVER)':'TILL TAP',                                                   
'DOCUMENT WORTHLESS ($200 & UNDER)':'DOCUMENT WORTHLESS','PICKPOCKET, ATTEMPT':'PICKPOCKET','GRAND THEFT / AUTO REPAIR':'THEFT',                                                   
'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS':'HUMAN TRAFFICKING','LEWD/LASCIVIOUS ACTS WITH CHILD':'CHILD','BIKE - ATTEMPTED STOLEN':'BIKE',                                                   
'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE':'HUMAN TRAFFICKING','CHILD PORNOGRAPHY':'CHILD','DISHONEST EMPLOYEE ATTEMPTED THEFT':'THEFT',                                                   
'FIREARMS RESTRAINING ORDER (FIREARMS RO)':'FIREARMS','VEHICLE - MOTORIZED SCOOTERS, BICYCLES, AND WHEELCHAIRS':'VEHICLE',                                                   
'FIREARMS TEMPORARY RESTRAINING ORDER (TEMP FIREARMS RO)':'FIREARMS'                                                   
})

In [15]:
#Change the dates collumn in LA data
DATE = []
for row in lapd['DATE OCC']:
    x = row.split(' ',1)
    DATE.append(x[0])
lapd.drop(columns ='DATE OCC')
DATE = pd.Series(DATE)
lapd['DATE OCC'] = DATE

In [16]:
#Used Excel to cut the years down to 2014-2017 for LA, and then narrowed that down to the most dangerous month (10/2017)
lapd = pd.read_csv("../crime_data/Cleaned_La_Crime_Data.csv")
LA_10_2017 = pd.read_csv("../crime_data/october_2017_LA_data.csv")

,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Premis Desc,Weapon Desc,LAT,LON
0,02/20/2010,1350,Newton,VIOLATION OF,SINGLE FAMILY DWELLING,NaN,33.9825,-118.2695
1,09/12/2010,45,Pacific,VANDALISM,STREET,NaN,33.9599,-118.3962
2,08/09/2010,1515,Newton,OTHER MISCELLANEOUS CRIME,ALLEY,NaN,34.0224,-118.2524
3,01/05/2010,150,Hollywood,VIOLATION OF,STREET,HAND GUN,34.1016,-118.3295
4,01/02/2010,2100,Central,RAPE,ALLEY,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",34.0387,-118.2488
...,...,...,...,...,...,...,...,...
2111674,03/28/2019,400,Hollywood,ARSON,SEX ORIENTED/BOOK STORE/STRIP CLUB/GENTLEMAN'S...,FIRE,34.0962,-118.3490
2111675,08/14/2019,1810,Wilshire,THEFT,STREET,NaN,34.0871,-118.3732
2111676,01/06/2019,2100,Olympic,CRIMINAL THREATS - NO WEAPON DISPLAYED,SIDEWALK,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",34.0637,-118.2870
2111677,10/16/2019,1800,Devonshire,THEFT,STREET,NaN,34.2266,-118.5085
